In [1]:
#libraries
import numpy as np
import matplotlib.pyplot as plt


$f(x) = 400 x_1^{2} + 0.004 x_2^{4}$

In [2]:
def evalf(x):
    assert type(x) is np.ndarray and len(x)==2
    return 400 * (x[0]**2) + 0.004 * (x[1]**4)

In [3]:
def evalg(x):
    assert type(x) is np.ndarray and len(x)==2
    return np.array([800 * x[0],0.016* x[1]**3])

In [4]:
# finding hessian matrix
def evalh(x):
    assert type(x) is np.ndarray and len(x)==2
    return np.array([[800,0],[0,0.048*(x[1]**2)]])

In [5]:
# finding invsere of hessian
def evalh_inv(x):
    assert  type(x) is np.ndarray and len(x)==2
    return np.linalg.inv(evalh(x))

In [6]:
# compute steplength by backtracking without scaling '
def compute_steplength_backtracking(x,gradf,alpha_start,rho,gamma):
    assert type(x) is np.ndarray and len(x)==2
    assert type(gradf) is np.ndarray and len(gradf) == 2
    assert type(alpha_start) is float and alpha_start>=0
    assert type(rho) is float and rho>=0
    assert type(gamma) is float and gamma >=0
    
    alpha = alpha_start
    while evalf(x-alpha*gradf)>evalf(x)-gamma*alpha*np.dot(gradf.T,gradf):
        alpha = rho * alpha
    return alpha

In [7]:
def compute_steplength_backtracking_scaled_direction(x,gradf,direction,alpha_start,rgho,gamma):
    assert type(x) is np.ndarray and len(x)==2
    assert type(gradf) is np.ndarray and len(gradf) == 2
    assert type(alpha_start) is float and alpha_start>=0
    assert type(rho) is float and rho>=0
    assert type(gamma) is float and gamma >=0
    assert type(direction) is np.ndarray and len(direction)==2
    
    alpha = alpha_start
    while evalf(x+alpha*direction)>evalf(x)+gamma*alpha*np.dot(gradf.T,direction):
        allpha = rho * alpha
    return alpha

In [8]:
NEWTON_CONSTANT_STEP_LENGTH = 1
NEWTON_BACKTRACKING_LINE_SEARCH = 2
WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH = 3
DIAG_MATRIX_SCALING_BACKTRACKING_LINE_SEARCH = 4

In [9]:
def find_minimizer_newton(x,tol,line_search_type,*args):
    assert type(x) is np.ndarray and len(x)==2
    assert type(tol) is float and tol>=0
    g_x = evalg(x)
    
    if line_search_type == NEWTON_BACKTRACKING_LINE_SEARCH :
        alpha_start = args[0]
        rho = args[1]
        gamma = args[2]
    if line_search_type == NEWTON_CONSTANT_STEP_LENGTH:
        k=0
        while np.linalg.norm(g_x)>tol:
            D_K = evalh_inv(x)
            direction = - np.matmul(D_K,g_x)
            step_length = 1
            x = np.add(x,np.multiply(step_length,direction))
            k+=1
            g_x = evalg(x)
    elif line_search_type == NEWTON_BACKTRACKING_LINE_SEARCH:
        k=0
        while (np.linalg.norm(g_x)>tol):
            D_K = evalh_inv(x)
            direction = -np.matmul(D_K,g_x)
            step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha_start,rho,gamma)
            x = np.add(x,np.multiply(step_length,direction))
            k+=1
            g_x = evalg(x)
    else:
        raise ValueError("Unknown line search type was entered")
    return x,evalf(x),k

3. [R] Consider $η
^k = 1$, ∀k = 1, 2, . . . in Algorithm 1. With starting point $x_
0 = (2, 2)$ and a stopping tolerance
$τ = 10^{−9}$
, find the number of iterations taken by the Newton’s method. Compare the number of iterations
with that taken by Newton’s method (with backtracking line search) in Algorithm 1. Note the minimizer and
minimum objective function value in each case. Comment on your observation

In [10]:
x_start=np.array([2,2])
tol = 1e-9

alpha_start = 1.0
rho = 0.5
gamma = 0.5

In [11]:
x,y,k = find_minimizer_newton(x_start,tol,NEWTON_CONSTANT_STEP_LENGTH)
print("For newtons method with constant step length ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For newtons method with constant step length 
No of iterations taken:  16
Minimizer:  [0.         0.00304488]
Minimum value: 3.4382653805813626e-13


In [12]:
x,y,k = find_minimizer_newton(x_start,tol,NEWTON_BACKTRACKING_LINE_SEARCH,alpha_start,rho,gamma)
print("For newtons method with backtracking line search ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For newtons method with backtracking line search 
No of iterations taken:  16
Minimizer:  [0.         0.00304488]
Minimum value: 3.4382653805813626e-13


* Looking at the solutions of both the methods we can observe that no of iterations is same and minimum value,minimizer is also same

4.[R] Compare the number of iterations obtained for the two variants of Newton’s method in the previous
question, with that of gradient descent algorithm (without scaling) with backtracking line search, gradient
descent algorithm (with scaling using a diagonal matrix) with backtracking line search (implemented in previous
labs), with starting point (2, 2). For backtracking line search, use $α_
0$ = 1, ρ = 0.5, γ = 0.5. Also compare the
minimizer and minimum objective function value in each case. Comment on your observations.

In [13]:
def cal_d_k(x):
    assert type(x) is np.ndarray and len(x)==2
    h = evalh(x)
    return np.array([[1/h[0][0],0],[0,1/h[1][1]]])

In [14]:
def find_minimizer(x,tol,line_search_type,*args):
    assert type(x) is np.ndarray and len(x)==2
    assert type(tol) is float and tol>=0
    g_x = evalg(x)
    alpha_start = args[0]
    rho = args[1]
    gamma =args[2]
        
    if line_search_type == WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH:
        K = 0
        while(np.linalg.norm(g_x) > tol):
            step_length = compute_steplength_backtracking(x,g_x,alpha_start,rho,gamma)
            x = np.add(x,np.multiply(step_length,g_x))
            K+=1
            g_x = evalg(x)
    elif line_search_type == DIAG_MATRIX_SCALING_BACKTRACKING_LINE_SEARCH:
        K = 0
        while (np.linalg.norm(g_x)>tol):
            D_K = cal_d_k(x)
            direction = -np.matmul(D_K,g_x)
            step_length = compute_steplength_backtracking_scaled_direction(x,g_x,direction,alpha_start,rho,gamma)
            x = np.add(x,np.multiply(step_length,direction))
            K+=1
            g_x = evalg(x)
    else:
        raise ValueError("Line search type is unknown")
    return x,evalf(x),k
    

In [15]:
my_start = np.array([2,2])

In [16]:
x,y,k=find_minimizer(my_start,tol,DIAG_MATRIX_SCALING_BACKTRACKING_LINE_SEARCH,alpha_start,rho,gamma)
print("For gradient descent algorithm with backtracking line search using diagonal matrix  ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

For gradient descent algorithm with backtracking line search using diagonal matrix  
No of iterations taken:  16
Minimizer:  [0.         0.00304488]
Minimum value: 3.4382653805813626e-13


* we can see gradient descent algorith with backtracking line search using diagonal matrix gives same values as question 3 this is the becausing hessian matrix turns out to be diagonal which is same as the daigonal matrix used in this method

In [ ]:
x,y,k = find_minimizer(my_start,tol,WITHOUT_SCALING_BACKTRACKING_LINE_SEARCH,alpha_start,rho,gamma)
print("For gradient descent algorithm without scaling backtracking line search ")
print("No of iterations taken: ",k)
print("Minimizer: ",x)
print("Minimum value:",y)

<ipython-input-2-97b3678e25e9>:3: RuntimeWarning: overflow encountered in double_scalars
  return 400 * (x[0]**2) + 0.004 * (x[1]**4)
<ipython-input-6-69bbc0807a69>:10: RuntimeWarning: invalid value encountered in double_scalars
  while evalf(x-alpha*gradf)>evalf(x)-gamma*alpha*np.dot(gradf.T,gradf):


* we can observe from above that gradient descent algorithm without scaling back tracking line search to longer time to  run so it is clearly not better than newtons method which takes only 16 iterations to find the minimum 